# Download Dataset

In [3]:
import os
import zipfile
import urllib.request

# URL for the dataset
dataset_url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
dataset_path = "ml-latest-small.zip"

# Download dataset
print("Downloading dataset...")
urllib.request.urlretrieve(dataset_url, dataset_path)
print("Download complete.")

# Extract dataset
print("Extracting files...")
with zipfile.ZipFile(dataset_path, "r") as zip_ref:
    zip_ref.extractall("movielens")
print("Extraction complete.")

# Check extracted files
print("Dataset files:", os.listdir("movielens/ml-latest-small"))


Download complete.
Extracting files...
Extraction complete.
Dataset files: ['links.csv', 'movies.csv', 'ratings.csv', 'README.txt', 'tags.csv']


# Load dataset

In [5]:
import pandas as pd

# Load datasets
movies = pd.read_csv("movielens/ml-latest-small/movies.csv")
ratings = pd.read_csv("movielens/ml-latest-small/ratings.csv")

# Display first few rows
print("\nMovies Dataset:")
print(movies.head())

print("\nRatings Dataset:")
print(ratings.head())



Movies Dataset:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings Dataset:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [7]:
print("Movies dataset shape:", movies.shape)
print("Ratings dataset shape:", ratings.shape)
print("Missing values:", movies.isnull().sum(), ratings.isnull().sum())


Movies dataset shape: (9742, 3)
Ratings dataset shape: (100836, 4)
Missing values: movieId    0
title      0
genres     0
dtype: int64 userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


# Convert Genre into Vector

use TF-IDF (Term Frequency-Inverse Document Frequency) to transform genres into numerical vectors.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert genres into TF-IDF vectors
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies["genres"].fillna(""))

# Convert to DataFrame
genre_df = pd.DataFrame(tfidf_matrix.toarray(), index=movies["title"])


# Compute Similarity Score

Now, compute similarity scores between movies.


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix)


# Recommend Movies Based on Similarity

In [13]:
def recommend_movies(title, num_recommendations=5):
    # Find index of the movie
    idx = movies[movies["title"] == title].index[0]
    
    # Get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]

    # Get movie titles
    movie_indices = [i[0] for i in sim_scores]
    return movies["title"].iloc[movie_indices]

# Example Usage
print("Recommended Movies:", recommend_movies("Toy Story (1995)"))


Recommended Movies: 1706                                       Antz (1998)
2355                                Toy Story 2 (1999)
2809    Adventures of Rocky and Bullwinkle, The (2000)
3000                  Emperor's New Groove, The (2000)
3568                             Monsters, Inc. (2001)
Name: title, dtype: object


create a user item-matrix

In [15]:
user_movie_matrix = ratings.pivot(index="userId", columns="movieId", values="rating").fillna(0)


# Build Neural Network Model

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense

num_users = ratings["userId"].nunique()
num_movies = ratings["movieId"].nunique()

# Define Input Layers
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

# Embedding Layers
user_embedding = Embedding(input_dim=num_users+1, output_dim=50)(user_input)
movie_embedding = Embedding(input_dim=num_movies+1, output_dim=50)(movie_input)

# Flatten Layers
user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

# Compute Dot Product (Similarity)
dot_product = Dot(axes=1)([user_vec, movie_vec])

# Output Layer
output = Dense(1, activation="linear")(dot_product)

# Build Model
model = Model([user_input, movie_input], output)
model.compile(optimizer="adam", loss="mse")

# Summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 50)     │     30,550 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 50)     │    486,250 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 50)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 50)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │          2 │ dot[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 516,802 (1.97 MB)

 Trainable params: 516,802 (1.97 MB)

 Non-trainable params: 0 (0.00 B)

# Train the model

In [33]:
# Prepare training data
user_data = ratings["userId"].values
movie_data = ratings["movieId"].values
ratings_data = ratings["rating"].values

# Train Model
model.fit([user_data, movie_data], ratings_data, epochs=5, batch_size=40)


Epoch 1/5
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - loss: 8.4766
Epoch 2/5
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.8502
Epoch 3/5
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.5405
Epoch 4/5
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.3660
Epoch 5/5
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.2555


# Predict Movie Ratings for a User

In [41]:
import numpy as np

# Ensure you have the same mappings used in training
user_mapping = {id_: idx for idx, id_ in enumerate(ratings["userId"].unique())}
movie_mapping = {id_: idx for idx, id_ in enumerate(ratings["movieId"].unique())}
reverse_movie_mapping = {idx: id_ for id_, idx in movie_mapping.items()}  # Reverse mapping for output

def recommend_for_user(user_id, num_recommendations=5):
    if user_id not in user_mapping:
        raise ValueError(f"User ID {user_id} not found in the dataset")

    mapped_user_id = user_mapping[user_id]  # Convert to zero-based index
    all_movies = np.array(list(movie_mapping.values()))  # Use mapped movie indices

    # Predict ratings
    predicted_ratings = model.predict([np.array([mapped_user_id] * len(all_movies)), all_movies])

    # Get top N recommendations
    top_indices = np.argsort(predicted_ratings.flatten())[-num_recommendations:][::-1]
    
    # Convert back to original movie IDs and get titles
    recommended_movie_ids = [reverse_movie_mapping[idx] for idx in all_movies[top_indices]]
    return movies[movies["movieId"].isin(recommended_movie_ids)]["title"].values

# Example Usage
print("Recommended Movies for User 1:", recommend_for_user(1))


304/304 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Recommended Movies for User 1: ["Kid in King Arthur's Court, A (1995)" 'Mystery Men (1999)']


# Hybrid Recommendation System

Combine Content-Based Filtering and Collaborative Filtering to make recommendations more accurate.

In [47]:
def hybrid_recommend(user_id, title, num_recommendations=5):
    # Get similar movies using content-based filtering
    content_recs = recommend_movies(title, num_recommendations)
    
    # Get user recommendations using collaborative filtering
    collaborative_recs = recommend_for_user(user_id, num_recommendations)
    
    # Combine results
    hybrid_recs = list(set(content_recs).union(set(collaborative_recs)))
    return hybrid_recs[:num_recommendations]

# Example Usage
print("Hybrid Recommendations:", hybrid_recommend(1, "Toy Story (1995)"))


Hybrid Recommendations: ["Emperor's New Groove, The (2000)", 'Monsters, Inc. (2001)', 'Antz (1998)', "Kid in King Arthur's Court, A (1995)", 'Adventures of Rocky and Bullwinkle, The (2000)']


# Recommendation 

In [58]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
movies_path = "C:/Users/harsh/Downloads/ml-latest-small/ml-latest-small/movies.csv"
ratings_path = "C:/Users/harsh/Downloads/ml-latest-small/ml-latest-small/ratings.csv"

movies = pd.read_csv(movies_path)
ratings = pd.read_csv(ratings_path)

# Process genres using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(movies['genres'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def get_recommendations_by_movie(movie_name):
    """Recommend movies based on movie name."""
    idx = movies[movies['title'].str.contains(movie_name, case=False, na=False)].index
    if len(idx) == 0:
        return ["No matching movie found"]
    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices].tolist()

def get_recommendations_by_genre(genre):
    """Recommend movies based on genre."""
    filtered_movies = movies[movies['genres'].str.contains(genre, case=False, na=False)]
    return filtered_movies['title'].head(5).tolist() if not filtered_movies.empty else ["No movies found"]

def get_recommendations_by_rating(threshold=4.0):
    """Recommend movies with average rating above a threshold."""
    avg_ratings = ratings.groupby('movieId')['rating'].mean()
    high_rated_movies = avg_ratings[avg_ratings >= threshold].index
    recommended_movies = movies[movies['movieId'].isin(high_rated_movies)].head(5)
    return recommended_movies['title'].tolist() if not recommended_movies.empty else ["No movies found"]

# GUI Setup
def recommend():
    user_input = entry.get().strip()
    criteria = criteria_var.get()
    if not user_input:
        messagebox.showwarning("Input Error", "Please enter a movie name, genre, or rating threshold.")
        return
    
    if criteria == "Movie Name":
        recommendations = get_recommendations_by_movie(user_input)
    elif criteria == "Genre":
        recommendations = get_recommendations_by_genre(user_input)
    elif criteria == "Rating" and user_input.replace('.', '', 1).isdigit():
        recommendations = get_recommendations_by_rating(float(user_input))
    else:
        messagebox.showerror("Input Error", "Invalid input. Please check your entry.")
        return
    
    result_label.config(text="\n".join(recommendations))

# Create GUI window
root = tk.Tk()
root.title("Movie Recommendation System")
root.geometry("500x400")

# Widgets
criteria_var = ttk.Combobox(root, values=["Movie Name", "Genre", "Rating"])
criteria_var.set("Movie Name")
criteria_var.pack(pady=10)

entry = tk.Entry(root, width=40)
entry.pack(pady=5)

recommend_button = tk.Button(root, text="Get Recommendations", command=recommend)
recommend_button.pack(pady=10)

result_label = tk.Label(root, text="", wraplength=400, justify="left")
result_label.pack(pady=10)

root.mainloop()
